In [11]:
# Import libaries and illustris library
import numpy as np
import illustris_python as il
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [ ]:
#
# everything in units of h! (h= 0.7)
#

In [12]:
# load data of citical points and segments from DisPerSE output

crits_file = './data/disperse/il_1_snap_135_subhalo_mask_75k75k19k.txt.NDnet_s5.97.up.NDskl.S500.BRK.ASMB.a.crits.txt'
crits_pos = np.loadtxt(crits_file, comments='#')[:,0:3]/0.7

segs_file = './data/disperse/il_1_snap_135_subhalo_mask_75k75k19k.txt.NDnet_s5.97.up.NDskl.S500.BRK.ASMB.a.segs.txt'
segs_pos = np.loadtxt(segs_file, comments='#')[:,0:3]/0.7
#print crits_pos, segs_pos, crits_pos.shape, segs_pos.shape

In [16]:
# load file with basic information:
# 0:HaloNumber, 1:SubhaloNumber, 2:StellarMass, 3:GasMass, 4:DarkMatterMass, 5-7:SubhaloPosition x,y,z
# 8-10:CoM Velocity x, CoM Velocity y, CoM Velocity z, 11-13:AngularMomentum x, AngularMomentum y, AngularMomentum z, 
# 14:Kappa, 15:Flatness, 16:Ellip, 17:Lambda

galaxy_file = './data/illustris/galaxyData_zeph_Il1_snap135.txt'
galaxies = np.genfromtxt(galaxy_file, comments='#')

In [47]:
# make_mask function takes position data
# pos_start is  index of first column with position data, s1-3 is box size constraints

# input: data: array 3 columns with x,y,z position
#        pos_start: first column index of position data in arrray
#        s1-3: box cutoff numbers

# output: binary mask with 1 or 0 for position whether it is in s1,s2,s3 or not

def make_mask(data, pos_start, s1, s2, s3):
    #create mask to constrain positions to within s1,s2,s3
    mask= ((data[:,pos_start] < s1) & (data[:,pos_start+1] < s2) & (data[:,pos_start+2] < s3)) 
    
    #return values that are within the s1-3 limit
    data_mask = data[mask]
    return data_mask

In [52]:
#calulate distance to a point:

#input: subhalo_mask: position of particles in column matrix x,y,z position
#        fil_pos_mat: position of filaments in column matrix x,y,z position

#output: array: 1st column is index of filament row that is nearest to particle. 2nd column is absolute distance

def distance_function(subhalo_mask, fil_pos_mat):
    
    #initialize number of columns in input file and number of steps to print
    subhalo_mask_size = subhalo_mask.shape[0]
    steps = max((subhalo_mask_size)/4,1)
    
    # create distance matrix - 2 columns for data 
    distance_mat = np.zeros(((subhalo_mask_size), 2))
    
    # enter for-loop which goes over the size of the rows of the position matrix 
    for i in range(0, subhalo_mask_size):
        
        #calculate raw position difference for 1 particle with ALL filament positions
        min_array = fil_pos_mat - subhalo_mask[i,0:3]
        
        #calculate actual distance
        min_dis = np.sqrt( (min_array*min_array).sum(axis=1) )
        
        #find minimum - i.e. nearest filament to particle
        min_index = np.argmin(min_dis, axis=0)
        
        #for each particle give index of filament point that is nearest and it´s absolute distance
        distance_mat[i,0] = min_index     
        distance_mat[i,1] = min_dis[min_index]
        
        if (i%steps == 0):
            print ("distane_function done with", (i/steps+1) *25,"%")

    # return the distance matrix
    return distance_mat

In [58]:
#calulate aligment of spin vector:

#input distance_mat: index of nearest segment point, distance, x,y,z of angular momentum vector) 
#       fil_pos_mat: column matrix x,y,z position

#output: array: 1st column is index of filament row that is nearest to particle. 2nd column is absolute distance, 
#                                              3rd is dot product of angular momentum vector with segment vector

def align_function(distance_mat, fil_pos_mat):
    
    #initialize number of columns in input file and number of steps to print
    distance_mat_size = distance_mat.shape[0]
    #steps = max((distance_mat_size)/4,1)
    
    #create align matrix - 3 columns for data 
    align_mat = np.zeros(((distance_mat_size), 3))
    
    # enter for-loop which goes over the size of the rows of the position matrix 
    for i in range(0, distance_mat_size):
        
        #calculate vector segment is pointing in (+1 from index in distance_mat to -1)
        fil_vec = fil_pos_mat[(int(distance_mat[i,0]) +1),0:3] - fil_pos_mat[(int(distance_mat[i,0]) -1),0:3]
        
        #get angular mom. vector of galaxy
        gal_vec = distance_mat[i,2:5]
        
        #calculate size
        fil_vec_size = np.sqrt( (fil_vec*fil_vec).sum(axis=0) )
        gal_vec_size = np.sqrt( (gal_vec*gal_vec).sum(axis=0) )
        
        align_mat[i,0] = distance_mat[i,0]
        align_mat[i,1] = distance_mat[i,1]
        
        align_mat[i,2] = abs(((fil_vec*gal_vec).sum(axis=0))/(fil_vec_size * gal_vec_size))

        #if (i%steps == 0):
         #   print "done with", i/steps *25,"%"

    # return the distance matrix 
    return align_mat

In [53]:
# filter galaxy data through mask and compute distance
# putting everything in units of h! (the inital file is) (h= 0.7)

index_pos_data = 5
galaxy_pos_mask = make_mask(galaxies, index_pos_data, 75000/0.7, 75000/0.7, 0.19*75000/0.7)

galaxies_positions = galaxy_pos_mask[:,5:8]

galaxy_segs_distance_mat = distance_function(galaxies_positions, segs_pos)
galaxy_crits_distance_mat = distance_function(galaxies_positions, crits_pos)

distane_function done with 25.0 %
distane_function done with 50.0 %
distane_function done with 75.0 %
distane_function done with 100.0 %
distane_function done with 25.0 %
distane_function done with 50.0 %
distane_function done with 75.0 %
distane_function done with 100.0 %


In [59]:
# Add the output of the distance function to the end of the matrix
# 0:HaloNumber, 1:SubhaloNumber, 2:StellarMass, 3:GasMass, 4:DarkMatterMass, 5-7:SubhaloPosition x,y,z
# 8-10:CoM Velocity x, CoM Velocity y, CoM Velocity z, 11-13:AngularMomentum x, AngularMomentum y, AngularMomentum z, 
# 14:Kappa, 15:Flatness, 16:Ellip, 17:Lambda 18: Index of segment/critical point, 19:absolute distance to filament

galaxy_crits_joined = np.concatenate(( galaxy_pos_mask[:,:], galaxy_crits_distance_mat[:,:]),axis=1)

galaxy_segs_joined = np.concatenate(( galaxy_pos_mask[:,:], galaxy_segs_distance_mat[:,:]), axis=1)

(724, 21)


In [ ]:
# create matrix for align function input

galaxy_segs_vec = galaxy_segs_joined[:,[18,19,11,12,13]]

gal_align_segs = align_function(galaxy_segs_vec, segs_pos)

# Add: 20:dot product ang. mom. vec. with filament vec.
galaxy_segs_data = np.concatenate((galaxy_segs_joined[:,:], gal_align_segs[:,2:3]), axis=1)

In [128]:
# 0:HaloNumber, 1:SubhaloNumber, 2:StellarMass, 3:GasMass, 4:DarkMatterMass, 5-7:SubhaloPosition x,y,z
# 8-10:CoM Velocity x, CoM Velocity y, CoM Velocity z, 11-13:AngularMomentum x, AngularMomentum y, AngularMomentum z, 
# 14:Kappa, 15:Flatness, 16:Ellip, 17:Lambda 18: Index of segment/critical point, 19:absolute distance to filament, 
# 20:dot product ang. mom. vec. with filament vec. 21-23: nearest segement coordinates 24: Index of nearest critical point, 
# 25: absolute distanceto crit., 26-28: nearest critical point coordinates
galaxy_data = np.zeros(((galaxy_segs_data.shape[0]), 29))

for i in range (0,galaxy_segs_data.shape[0]):
    galaxy_data[i,0:21] = galaxy_segs_data[i,:]
    galaxy_data[i,21:24] = segs_pos[int(galaxy_segs_data[i,18]),:]
    galaxy_data[i,24:26] = galaxy_crits_distance_mat[i,:]
    galaxy_data[i,26:29] = crits_pos[int(galaxy_data[i,24]),:]

In [129]:
galaxy_data.shape

(724, 29)

In [131]:
np.savetxt('./data/galaxy_data_1.txt', galaxy_data, delimiter =' ', 
           header = '0:HaloNumber, 1:SubhaloNumber, 2:StellarMass, 3:GasMass, 4:DarkMatterMass, 5-7:SubhaloPosition x, SubhaloPosition y, SubhaloPosition z, 8-10:CoM Velocity x, CoM Velocity y, CoM Velocity z, 11-13:AngularMomentum x, AngularMomentum y, AngularMomentum z, 14:Kappa, 15:Flatness, 16:Ellip, 17:Lambda, 18: Index of nearest segment, 19:absolute distance from galaxy to nearest segment point 20:dot product of angular momentum vector with segment vector 21-23: nearest segement coordinates 24: Index of nearest critical point, 25: absolute distance from galaxy to nearest critical point 26-28: nearest critical point coordinates'
           )